In [1]:
from GUI import GUI
import layout.lib.repetition_recognition as rep

In [56]:
gui = GUI(img_file='data/input/v2.jpg')

In [57]:
gui.detect_element()
# gui.load_detection_result()
gui.visualize_element_detection()

[Text Detection Completed in 0.974 s] Input: data/input/v2.jpg Output: data/output\ocr\v2.json
[Compo Detection Completed in 0.594 s] Input: data/input/v2.jpg Output: data/output\ip\v2.json
[Merge Completed] Input: data/input/v2.jpg Output: data/output\uied\v2.jpg


In [58]:
gui.recognize_layout()
gui.visualize_layout_recognition()

[Layout Recognition Completed in 1.425 s] Input: data/input/v2.jpg Output: data/output\layout\v2.json


In [85]:
gui.visualize_element_detection()

In [70]:
gui.visualize_layout_recognition()

In [5]:
gui.cvt_compos_json_to_dataframe()

In [6]:
gui.compos_df.recognize_element_groups_by_clustering()

In [10]:
b= gui.compos_df.visualize_fill(gather_attr='group')

In [8]:
gui.compos_df.calc_gap_in_group()

In [11]:
import layout.lib.repetition_recognition as rep
gui.compos_df.compos_dataframe = rep.recog_repetition_text(gui.compos_df, show=True, only_non_contained_compo=True)

In [12]:
b = gui.compos_df.visualize_fill(gather_attr='group_text')

In [11]:
df = gui.compos_df.compos_dataframe

In [12]:
df[df['group'] == 't-2']

,id,class,column_min,column_max,row_min,row_max,height,width,area,center,center_column,center_row,text_content,children,parent,alignment_in_group,gap,group
18,18,Text,189,454,216,260,44,265,11660,"(321.5, 238.0)",321.5,238.0,BonTon : Pink Rose Dresses\n$ 17 Shipped,-1,-1,v,197,t-2
19,19,Text,188,444,457,475,18,256,4608,"(316.0, 466.0)",316.0,466.0,WHBM : Extra 40 % Off Sale,-1,-1,v,225,t-2
20,20,Text,189,460,700,747,47,271,12737,"(324.5, 723.5)",324.5,723.5,12 Ribbed Cotton Tank Tops\n$ 25 Shipped,-1,-1,v,-1,t-2


In [12]:
from sklearn.cluster import DBSCAN
import numpy as np

In [13]:
group_compos = df[df['group'] == 't-3']
if group_compos.iloc[0]['alignment_in_group'] == 'v':
    group_compos = group_compos.sort_values('center_row')
else:
    group_compos = group_compos.sort_values('center_column')
gaps = list(group_compos['gap'])

clustering = DBSCAN(eps=10, min_samples=1).fit(np.reshape(gaps[:-1], (-1, 1)))
gap_labels = list(clustering.labels_)
gap_label_count = dict((i, gap_labels.count(i)) for i in gap_labels)  # {label: frequency of label}

In [14]:
gap_label_count

{0: 1, 1: 4, 2: 1}

In [15]:
gap_labels

[0, 1, 1, 1, 1, 2]

In [23]:
group_compos

,id,class,column_min,column_max,row_min,row_max,height,width,area,center,center_column,center_row,text_content,children,parent,alignment_in_group,gap,group
16,16,Text,2,26,45,59,14,24,336,"(14.0, 52.0)",14.0,52.0,( E,-1,-1,v,101,t-3
19,19,Text,28,147,160,175,15,119,1785,"(87.5, 167.5)",87.5,167.5,Search Foods,-1,-1,v,76,t-3
20,20,Text,24,108,251,264,13,84,1092,"(66.0, 257.5)",66.0,257.5,+ Breakfast,-1,-1,v,68,t-3
21,21,Text,23,80,332,345,13,57,741,"(51.5, 338.5)",51.5,338.5,+ Lunch,-1,-1,v,68,t-3
22,22,Text,23,84,413,426,13,61,793,"(53.5, 419.5)",53.5,419.5,+ Dinner,-1,-1,v,68,t-3
23,23,Text,24,81,494,506,12,57,684,"(52.5, 500.0)",52.5,500.0,+ Snack,-1,-1,v,44,t-3
24,24,Text,17,64,550,563,13,47,611,"(40.5, 556.5)",40.5,556.5,Totals,-1,-1,v,-1,t-3


In [18]:
gaps = list(group_compos['gap'])

In [20]:
gaps

[101, 76, 68, 68, 68, 44, -1]

In [ ]:
for i in range(len)